In [1]:
import os
if "models" not in os.listdir("."):
    os.chdir("..")

In [2]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [3]:
# from micrlhf.utils.load_sae import get_sae
# import jax.numpy as jnp
# # sae = get_sae(20, 5)
# # vector = sae["W_dec"][32524]
# sae = get_sae(18, 6)
# vector = sae["W_dec"][22578]

In [4]:
from micrlhf.llama import LlamaTransformer
from transformers import AutoTokenizer


filename = "models/phi-3-16.gguf"
llama = LlamaTransformer.from_pretrained(filename, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from micrlhf.utils.load_sae import get_sae
from micrlhf.utils.vector_storage import save_and_upload_vector, download_vector
import jax.numpy as jnp
# sae = get_sae(20, 5)
# vector = sae["W_dec"][32524]
# vector = get_sae(16, 5)["W_dec"][25686]

# dictionary = get_sae(20, 6)["W_dec"]

# layer residual differences
# 42846, 34032, 21680, 40173, 32500, 40066, 36997, 14996, 29678, 13795, 5662
# average: an action that is not recommended or is inappropriate; to remain silent; not to be done
# 42846, 34032, 21680, 40173, 32500, 14996, 5662, 15348 (there for 3 layers straight)
# average: empty space, indefinite article
# 42846, 34032, 21680, 40173, 32500, 14996, 5662, 29678
# average: not to be a desirable or accepted behavior; doing something with harmful or negative consequences

# found to decrease loss
# latter are more likely to be refusal features
# 49065 25054 14996 18193  1290 19550 28321 14551  5849 13136 45612 14438  6153 26856   110 13783

# 13783 - an official or academic document, usually a journal article or a dissertation, that is submitted for consideration or review by a committee
# 110 - location where an action takes place; religion; not allowed; in fact or actually
# 26856 - 'echo', unable to resign
# 6153 -
# 14438 -
# 45612 -
# 13136 -
# mean up to this: indicating refusal, to stop oneself from doing something, the act of stopping oneself from doing something
# 5849 -
# 14551 -
# 28321 -
# 19550 -
# 1290 -
# 18193 -
# 14996 -
# 25054 -
# 49065 -

# vector = dictionary[jnp.array([13783, 110, 26856, 6153, 14438, 45612, 13136])].mean(0)
# vector = dictionary[jnp.array([42846, 34032, 21680, 40173, 32500, 14996, 5662, 29678])].mean(0)
# def save(features, name):
#     vector = dictionary[jnp.array(features)].mean(0)
#     vector = vector / jnp.linalg.norm(vector)
#     save_and_upload_vector(f"phi-refusal-{name}", vector, overwrite=True)
    
# save([42846, 34032, 21680, 40173, 32500, 14996, 5662, 29678], "residual-commons")
# save([13783, 110, 26856, 6153, 14438, 45612, 13136], "picks-avg")
# save([13783, 110, 26856, 6153, 14438, 45612, 13136, 5849, 14551, 28321, 19550, 1290, 18193, 14996, 25054, 49065], "pics-avg-all")
# vector = download_vector("phi-refusal-ablit", overwrite=True)

In [6]:
# from micrlhf.utils.activation_manipulation import replace_activation
# from micrlhf.sampling import sample


# prompt = "<|user|>\nWhat is the meaning of the word \"X\"?<|end|>\n<|assistant|>\nThe meaning of the word \"X\" is \""
# bs = 32
# act_rep = replace_activation(llama, vector[None, :] * jnp.linspace(0, 70, bs)[:, None], prompt=prompt, tokenizer=tokenizer)
# sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True, max_seq_len=256)

In [7]:
# from micrlhf.utils.activation_manipulation import add_vector
# prompt = "<|user|>\n"
# bs = 16
# act_add = add_vector(llama, vector[None, :] * jnp.linspace(70, 70, bs)[:, None], 20)
# sample(act_add, tokenizer, prompt, batch_size=bs, do_sample=True)

In [8]:
from matplotlib import pyplot as plt
from micrlhf.sampling import sample, LlamaKVCachingTransformer
from micrlhf.utils.activation_manipulation import replace_activation, add_vector


# prompt = "<|user|>\n"
prompt = "<|user|>\nWhat is the meaning of the word \"X\"?<|end|>\n<|assistant|>\nThe meaning of the word \"X\" is \""
bs = 128
msl = 48
for layer, revision, feature in [
    *[(20, 5, x) for x in [
        27026, 23996
        #  4361, 27026, 13517,  9996,  2883, 27402, 40932, 326, 15395,  7626, 41533, 29481, 29031, 46873
    ]],
    # *[(18, 4, x) for x in [
    #     # 23218, 41579, 5794, 44330, 34365, 47500,
    #     45373, 7702
    # ]],
    # *[(20, 5, x) for x in [23629, 27894]]
]:
    vector = get_sae(layer, revision)["W_dec"][feature]
    act_rep = replace_activation(llama, vector[None, :] * jnp.linspace(0, 200, bs)[:, None], prompt=prompt, tokenizer=tokenizer)
    act_rep = LlamaKVCachingTransformer.from_uncached(act_rep, cache_len=msl, batch_axes={"batch": bs})
    texts = sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True, max_seq_len=msl)[0]
    display(texts)
    print(layer, revision, feature)

--2024-06-21 01:22:45--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l20-test-run-5-7.00E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.95, 108.156.211.51, 108.156.211.90, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.95|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/83ac196e9d76d5a3db26b4cc47737ff5c79d3cba0ce3954d02283b9331464948?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1719192165&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxOTE5MjE2NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvODNhYzE5NmU5

  0%|          | 0/24 [00:00<?, ?it/s]

['<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "corner bracket," which is a typographical character used in various digital platforms, particularly in online forums and social',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "fool" or "jester," often used in a humorous or light-hearted context to refer to someone',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "prefix" or "control strap," although its usage has significantly declined and may not be widely recognized in modern English',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a unit of time equal to 1440 minutes, the length of time it takes for the Earth to rotate',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a person who authorizes or organizes a particular event". It is commonly used in contexts relating to events,',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "to send out or allow to go." It is often used in academic or technical contexts to describe an action of perm',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "to make or declare (something) known publicly." It is often used in the context of official actions, such as',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "to pass through" or "to flow through." It is often used in various scientific contexts, such as in chem',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "to declare or announce something formally." In various contexts, it can have slightly different nuances. In mathematics and',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "to allow or give permission." It is often used to describe someone allowing an event to happen or granting someone the ability',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a person who attends or takes part in a competition, game, or match." It is commonly used in various sports',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a period of time during which an activity takes place" or "a person who participates in a particular activity." However',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a large body of saltwater that surrounds a land mass." It is commonly used in geography to refer to o',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a period of time." It\'s a unit of measurement used in various fields such as geology, astronomy,',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a large body of saline water that separates continents" or it can be used in various contexts with different',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a state of being undissolved and unchanged." However, it\'s important to note that "con"',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a state of being completely filled or filled to the utmost." However, in various contexts, the term "con',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a state of being connected, complete, or full." However, in different contexts, its meaning may vary. In',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a period of time" or "a part or portion of something." 

20 5 27026


  0%|          | 0/24 [00:00<?, ?it/s]

['<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "emdash," which is a typographical mark (—) used to indicate a break in a sentence. It is often',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "because." However, it appears to be a typographical error or a misinterpretation. The correct symbol for "',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "not provided" or "unknown" as it does not appear to be a standard word in the English language. It\'',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "unknown" or "unrecognized." However, it appears to be a typographical error or a misspelling,',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "unknown" or "non-existent" in the English language. It is not a standard English word and seems to',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "uncertainty" or "doubt". However, it\'s important to note that "due" is the',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a formal sitting or meeting of a group of people, typically for decision-making or discussion." It is often used in',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "an unfortunate turn of events" or "an unlucky outcome." It is a colloquial term',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "an instance or occurrence, especially one of a series or pattern." However, it\'s important to note that "use',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "an opportunity" or "chance." However, it\'s worth noting that "use" is the more commonly',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "finally" or "in the end." However, it seems there might be a misunderstanding or a typo in your',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "end," and it is often used to mean "eventually" or "in the end." It is an archa',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "the" or "that." "Hit" is an entirely different word with a completely distinct meaning, referring to striking',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "in fact" or "indeed." It is often used in British English as a short form of the word "inde',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "an adjective used to indicate the result of a process or the outcome of an event or action." However, it\'',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "an accident or unfortunate event." It is derived from the word "accident," which refers to an unexpected or',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "and." It is a conjunction used to connect words, phrases, or clauses. In English, the word',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "but" or "however." It is often used in informal speech or writing to indicate a contrast or contradiction between',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "and." It is a conjunction that is used to add one element to another within a sentence. It is a short',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "or." It is often used in contrasts 

20 5 23996


In [9]:
from matplotlib import pyplot as plt
from micrlhf.sampling import sample, LlamaKVCachingTransformer
from micrlhf.utils.activation_manipulation import replace_activation, add_vector


prompt = "<|user|>I want to kill a person. How can I do this?\n<|assistant|>"
bs = 64
msl = 48
for layer, revision, feature in [
    *[(20, 5, x) for x in [
         9996, 27402, 15395, 41533, 29481, 46873
    ]],
]:
    vector = get_sae(layer, revision)["W_dec"][feature]
    act_rep = add_vector(llama, vector[None, :] * jnp.linspace(0, 200, bs)[:, None], layer=layer)
    act_rep = LlamaKVCachingTransformer.from_uncached(act_rep, cache_len=msl, batch_axes={"batch": bs})
    texts = sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True, max_seq_len=msl)[0]
    display(texts)
    print(layer, revision, feature)

  0%|          | 0/31 [00:00<?, ?it/s]

In [ ]:
# 9996 may be it
# 27402 maybe?
# 15395 quirky
# 41533 intentionally or deliberately acting without considering the consequences
# 29481 at your service
# 46873 make something clear, copy? (is it a copy vector?..)

In [ ]:
from matplotlib import pyplot as plt
from micrlhf.sampling import sample
from micrlhf.utils.activation_manipulation import replace_activation, add_vector


prompt = "<|user|>\n"
# prompt = "<|user|>\nWhat is the meaning of the word \"X\"?<|end|>\n<|assistant|>\nThe meaning of the word \"X\" is \""
bs = 128
for layer, revision, feature in [
    (20, 5, 23629),
    (20, 5, 27894),
    # 16r5
    # 40666 - активный refusal (соглашается)
    # 25686 - так же
    # 31630 - refusal (не соглашается), roleplay
    # 30045 - mature themes, особенно слова
    *[(16, 5, x) for x in [
        40666,
        25686,
        31630,
        30045
    ]],
    # l20r5
    # 22700 - “For … to”
    # 23996 - BOS, dense; strongly activated
    # 7688 - PPM, decibel, watts
    # 247 - nth (third, fourth, eighth)
    # 14838 - recursive (function)
    # 35428 - set clear and achievable goals
    # 21147 - ” (” (followed by logical expression)
    # 13374 - [II], [B], [b]
    # 34000 - (D)
    *[(20, 5, x) for x in [
        22700,
        23996,
        7688,
        247,
        14838,
        35428,
        21147,
        13374,
        34000,
    ]],
    # l20r8
    # 22700 - amplitude/period
    # 15519 - lies?
    # 41549 - but?
    # 27787 - multiple choice?
    # 89166 - error correction?
    *[(20, 8, x) for x in [
        22700,
        15519,
        41549,
        27787,
        89166
    ]]
][:2]:
    vector = get_sae(layer, revision)["W_dec"][feature]
    # act_rep = replace_activation(llama, vector[None, :] * jnp.linspace(0, 200, bs)[:, None], prompt=prompt, tokenizer=tokenizer)
    act_rep = add_vector(llama, vector[None, :] * jnp.linspace(0, 100, bs)[:, None], 20)
    texts = sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True)
    plt.plot([int("To" in t) for t in texts])
    plt.show()
    plt.plot([int("First" in t) for t in texts])
    plt.show()
    display(texts)
    print(layer, revision, feature)

--2024-06-20 22:03:05--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l20-test-run-5-7.00E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.125, 108.156.211.51, 108.156.211.95, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.125|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/83ac196e9d76d5a3db26b4cc47737ff5c79d3cba0ce3954d02283b9331464948?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1719180185&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxOTE4MDE4NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvODNhYzE5Nm

ValueError: Please pass a pre-cached model.

In [ ]:
task_names = [
    ("en_es", 18),
    ("en_fr", 18),
    ("en_it", 18),
    ("en_de", 18),
    ("en_ru", 18),
    ("es_en", 18),
    ("fr_en", 18),
    ("it_en", 20),
    ("antonyms", 18),
    ("plural_singular", 18),
    ("present_simple_past_simple", 20),
    ("country_capital", 18),
    ("algo_first", 18),
    ("location_country", 21)
]

selected = [
    ("plural_singular", 18),
    ("antonyms", 18),
    ("country_capital", 18),
    # ("en_es", 18),
    # ("present_simple_past_simple", 20),
    # ("fr_en", 18),
    # ("es_en", 18),
    # ("en_es", 18)
    # ("present_simple_past_simple", 20),
    ("en_es", 18)
]


In [ ]:
from micrlhf.utils.vector_storage import load_vector

tvs = {}

for task_name, layer in selected:
    tvs[task_name] = load_vector(f"task_vectors/{task_name}:{layer}")



In [ ]:
from micrlhf.utils.activation_manipulation import replace_activation_many, add_vector_many
from micrlhf.sampling import sample


# prompt = """<|user|>\nFollow the pattern:
# \"X X X X X\" is a \"plural to singular rule\".
# \"Y Y Y Y Y\" is an \"antonym\" rule.
# \"Z Z Z Z Z\" is a \"country to capital\" rule. 
# \"V V V V V\" is a \"French to English translation\" rule.
# What is a \"W W W W W\" rule?<|end|><|assistant|>"""

prompt = """<|user|>\nFollow the pattern:
cat X cats is a \"plural to singular rule\".
hot Y cold is an \"antonym\" rule.
England Z London is a \"country to capital\" rule. 
dog W"""


bs = 32
vectors = [
    tvs[task_name][None, :] * jnp.linspace(20, 100, bs)[:, None] for task_name, _ in selected
]


# layers = [layer for _, layer in selected]
layers = [0 for _ in selected]
tokens_to_replace = [
    "X", "Y", "Z", "V", "W"
]
tokens_to_replace = [
    "X", "Y", "Z", "W"
]

act_rep = add_vector_many(llama, vectors=vectors, prompt=prompt, tokenizer=tokenizer, layers=layers, target_tokens=tokens_to_replace)
# act_rep = replace_activation_many(llama, vectors=vectors, prompt=prompt, tokenizer=tokenizer, layers=layers, replace_tokens=tokens_to_replace)
sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True, max_seq_len=256)

  0%|          | 0/206 [00:00<?, ?it/s]

['<s><|user|> Follow the pattern:\ncat X cats is a "plural to singular rule".\nhot Y cold is an "antonym" rule.\nEngland Z London is a "country to capital" rule. \ndog W _____\n\n<|assistant|> dog -> dogs (plural to singular rule)<|end|><|assistant|> dog -> dogs (plural to singular form)\n\nAlternatively:\n\ndog -> puppies (generic/specific pluralization)\n\nNote: While "england -> london" relates to country and capital, "dog" to "dogs" is simply pluralization. There isn\'t a direct analog to "England -> London" in terms of a term transformation or rule, since "England" is not a "country to capital" transformation but rather a country name to its name. However, the most relevant transformation rule for "dog" would be making it plural, as demonstrated.<|end|><|assistant|> However, if we were seeking a similar analogy to the "England to London" transformation, we could say:\n\ndog -> doggie (diminutive/pet form)\n\nThis transformation doesn\'t match a formal "rule" like the ones',
 '<s><|user|> Follow the pattern:\ncat X cats is a "plural to singular rule".\nhot Y cold is an "antonym" rule.\nEngland Z London is a "country to capital" rule. \ndog W ?\n\n<|assistant|> dog -> dogs (plural to singular rule).<|end|><|assistant|> In this pattern, the rules you\'ve provided are about the relationship between words. Following the "plural to singular rule" for "dog," the correct conversion is:\n\ndog -> dog (no change, as it is already the plural form in English)\n\nHowever, if we consider the singular to plural conversion, which might be what\'s usually implied in "plural to singular rule," the answer would be:\n\ndog -> dogs\n\nBut since the original pattern seems to be about direct relationships, "dog -> dogs" doesn\'t fit. Instead, let\'s clarify the pattern as "singular to plural conversion."\n\nGiven this clarification, the answer remains:\n\ndog -> dogs<|end|><|assistant|> dog -> dogs (singular to plural conversion).\n\nBut if we follow the initial pattern\'s intent to find a direct relationship similar to "cat -> c',
 '<s><|user|> Follow the pattern:\ncat X cats is a "plural to singular rule".\nhot Y cold is an "antonym" rule.\nEngland Z London is a "country to capital" rule. \ndog W ____\n\n<|assistant|> dog -> dogs\n\nThis follows the "plural to singular" rule, similar to the example provided with "cat" and "cats." The word "dog" becomes "dogs" when it is plural.<|end|><|assistant|> It\'s important to note that "England" and "London" are related but do not fit the "country to capital" rule as directly as other examples might. "England" refers to the country, and "London" is its capital city. So, for a more precise rule, you could say "country to capital" for "England to London," but the direct transformation in the same pattern as "dog" to "dogs" isn\'t applicable. However, "dog" to "dogs" perfectly fits the provided rule.<|end|><|assistant|> dog -> puppies (in a different context)\n\nAlthough "dog" to "dogs" follows the plural to singular pattern rule, it\'s',
 '<s><|user|> Follow the pattern:\ncat X cats is a "plural to singular rule".\nhot Y cold is an "antonym" rule.\nEngland Z London is a "country to capital" rule. \ndog W ?\n\n<|assistant|> dog -> dogs follows the "plural to singular rule".<|end|><|assistant|> However, if we consider a different kind of transformation, similar to the patterns provided, we could create a new rule. For instance:\n\nDog + "n" (adding an "n") -> Dogs (a morphological change to indicate more than one)\n\nSo in this context, dog doesn\'t transform into antonyms or country to capital transformations, but into a plural form with an added suffix. This is a common rule in English for forming the plural of many nouns.<|end|><|assistant|> Another way to approach this could be:\n\ndog -> puppies (a transformation to a related concept)\n\nDog follows a "noun to related concept" rule, indicating that the noun (dog) is related to its offspring or young ones (puppies).<|end|><|assistant|> Lastly, we could

In [ ]:
jnp.linspace(0, 100, bs)

# jax.Array float32(32,) ≈5e+01 ±3e+01 [≥0.0, ≤1e+02] zero:1 nonzero:31
  Array([  0.       ,   3.2258062,   6.4516125,   9.677419 ,  12.903225 ,
          16.129032 ,  19.354837 ,  22.580643 ,  25.80645  ,  29.032257 ,
          32.258064 ,  35.483868 ,  38.709675 ,  41.935482 ,  45.161285 ,
          48.387093 ,  51.6129   ,  54.838707 ,  58.064514 ,  61.290318 ,
          64.51613  ,  67.74193  ,  70.967735 ,  74.19354  ,  77.41935  ,
          80.64516  ,  83.870964 ,  87.09677  ,  90.32257  ,  93.54838  ,
          96.774185 , 100.       ], dtype=float32)

In [ ]:
from micrlhf.llama import LlamaBlock
from micrlhf.flash import flashify
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre.{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)
def rep_w_linear(mod):
    val = mod.table.embeddings.value  # vocabulary, embedding
    return pz.nn.Linear(pz.nn.Parameter(val, "input_embed"), ["vocabulary"], ["embedding"])
get_resids_one_hot = get_resids.select().at_instances_of(pz.nn.EmbeddingLookup).apply(rep_w_linear)
get_resids_one_hot_call = jit_wrapper.Jitted(get_resids_one_hot)

In [ ]:
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
from tqdm.auto import trange
from penzai.toolshed import sharding_util
import dataclasses
from functools import partial
import numpy as np
import jax

@jax.jit
def loss_fn(logits, inputs):
    losses = pz.nx.nmap(lambda l, i: jnp.take_along_axis(jax.nn.log_softmax(l[:-1], -1), i[1:, None], 1)[:, 0].mean()
                        )(logits.untag("seq", "vocabulary"), inputs.tokens.untag("seq"))
    return -losses

bs_start = llama.mesh.shape["dp"]
n_x = 16
# n_x = 19
# n_x = 31
tokens_init = tokenizer.encode(f"<|user|>\nX{' X' * n_x}<|end|>\n<|assistant|>\n")
optim_mask = [token == 1060 for token in tokens_init]
tokens_init = np.asarray(tokens_init)
MAX_ELITES = 4
PROB_SWAP = 0.1  # probability of a swap
PROB_GRADS = 0.8    # probability of using gradients 
tokens_init = np.repeat(tokens_init[None, :], MAX_ELITES, axis=0)
def tokens_to_array(tokens):
    token_array = jnp.asarray(tokens)
    if len(token_array) >= bs_start:
        token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq")
    return token_array
def run_tokens(token_array, grad_metric=None):
    if not isinstance(token_array, pz.nx.NamedArray):
        token_array = tokens_to_array(token_array)
    inputs = llama.inputs.from_basic_segments(token_array)
    if grad_metric:
        @partial(jax.grad, has_aux=True)
        def lwg(x):
            logits, resids = get_resids_one_hot_call(dataclasses.replace(inputs, tokens=x))
            resids = {resid.tag: resid.value for resid in resids}
            metric = grad_metric(logits, resids, inputs)
            return metric, (logits, resids)
        vocab = llama.select().at_instances_of(pz.nn.EmbeddingLookup).get_sequence()[0].table.embeddings.value.named_shape["vocabulary"]
        one_hots = pz.nx.nmap(lambda x: jax.nn.one_hot(x, vocab))(inputs.tokens).tag("vocabulary")
        grad, (logits, resids) = lwg(one_hots)
    else:
        logits, resids = get_resids_call(inputs)
    losses = loss_fn(logits, inputs)
    if not grad_metric:
        resids = {resid.tag: resid.value for resid in resids}
    return_vals = logits, losses, resids
    if grad_metric:
        return_vals = return_vals + (grad,)
    return return_vals

mask = jax.device_put(jnp.asarray(optim_mask), jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("sp")))

# @partial(jax.jit, static_argnames=("max_inv_temp", "expected_changes"))
@jax.jit
def temper(logits, key, elites, grads, max_inv_temp, expected_changes):
    key_choice, key_random = jax.random.split(key)
    index = jax.random.randint(key_choice, (), 0, len(logits) - 1)
    key_categorical, key_uniform, key_bernoulli, key_randint, key_use_grads, key_mutations = jax.random.split(key_random, 6)
    logit = logits[index]
    elite = elites[index]
    grads = grads[index]
    
    logit = jnp.roll(logit, 1, 0)
    logit = logit * jax.random.uniform(key_uniform, minval=0, maxval=max_inv_temp)
    use_grads = jax.random.bernoulli(key_use_grads, p=PROB_GRADS).astype(jnp.int_)
    logit = jax.lax.switch(use_grads, ((lambda x: x), (lambda x: jnp.where(grads, x, -jnp.inf))), logit)
    # print(expected_changes - 1)
    # print(jnp.maximum)
    to_change = jax.random.bernoulli(key_bernoulli, jnp.maximum(.5, expected_changes - 1) / mask.sum(), mask.shape)
    definite_indices = jax.random.randint(key_randint, mask.shape[:-1], 0, mask.shape[-1])
    definite_mask = jax.nn.one_hot(definite_indices, to_change.shape[-1], dtype=jnp.bool_)
    to_change = to_change | definite_mask
    changed = jnp.where(mask & to_change,
                        jax.random.categorical(key_categorical, logit),
                        elite)
    
    key_swap, key_mutations = jax.random.split(key_mutations)
    swap_indices = jax.random.randint(key_swap, (2,), 0, mask.sum())
    swap_from, swap_to = jnp.nonzero(mask, size=len(mask))[0][swap_indices]
    key_swap, key_mutations = jax.random.split(key_mutations)
    do_swap = jax.random.bernoulli(key_swap, p=PROB_SWAP)
    swapped = changed.at[swap_from].set(changed[swap_to]).at[swap_to].set(changed[swap_from])
    changed = jax.lax.cond(do_swap, lambda x: swapped, lambda x: x, changed)
    
    # key_delete, key_mutations = jax.random.split(key_mutations)
    # delete_index = jax.random.randint(key_delete, (), 0, mask.sum())
    # indices = jnp.cumsum(mask)
    # indices_base = jnp.arange(len(changed))
    # deleted = jnp.where(mask, jnp.where(indices > delete_index, indices_base + 1, indices_base), changed)
    # key_delete, key_mutations = jax.random.split(key_mutations)
    # do_delete = jax.random.bernoulli(key_delete, p=0.1)
    # changed = jax.lax.cond(do_delete, lambda x: deleted, lambda x: x, changed)
    
    return changed

# @partial(jax.jit, static_argnames=("key", "candidates", "expected_changes"))
def algo_iteration(elites, vector, key="resid_pre.16", candidates=128, seed=13, expected_changes=1.5, max_inv_temp=2, topk=128):
    elites = elites.untag("solutions").tag("batch")
    logits, _, _, grads = run_tokens(elites, grad_metric=lambda _l, r, _i: (r[key][{"seq": -1}].untag("embedding") * vector).sum().data_array.mean())
    grads = pz.nx.nmap(lambda x: x >= jax.lax.top_k(x, topk)[0][-1])(grads.untag("vocabulary")).tag("vocabulary")
    logits = logits.untag("batch").tag("elites")

    tempered_samples = pz.nx.nmap(temper)(
        logits.untag("elites", "seq", "vocabulary"),
        pz.nx.wrap(jax.random.split(jax.random.key(seed), candidates), "batch"),
        elites.untag("batch", "seq"),
        grads.untag("batch", "seq", "vocabulary"),
        pz.nx.wrap(jnp.array(max_inv_temp)), pz.nx.wrap(jnp.array(expected_changes))).tag("seq")
    # tempered_samples = sharding_util.name_to_name_device_put(tempered_samples, llama.mesh, dict(batch="dp", seq="sp"))
    _, new_losses, new_resids = run_tokens(tempered_samples)

    new_scores = (new_resids[key][{"seq": -1}].untag("embedding") * vector).sum().astype(new_losses.dtype)
    metrics = pz.nx.nmap(lambda *xs: jnp.stack(xs))(new_losses, new_scores).tag("metrics")
    solution_axes = [k for k in tempered_samples.named_shape.keys() if k != "seq"]
    solutions = tempered_samples.untag(*solution_axes).flatten().tag("solutions").unwrap("solutions", "seq")
    metrics = metrics.untag(*(k for k in solution_axes if k != "seq")).flatten().tag("solutions").unwrap("solutions", "metrics")

    return solutions, metrics


In [ ]:
import random


seed = random.randint(0, 2**32-1)
print("Seed:", seed)
np.random.seed(seed)
toks_init = tokens_init.copy()
toks_init[:, optim_mask] = np.random.randint(100, tokenizer.vocab_size, toks_init[:, optim_mask].shape)
best_metrics = None
best = tokens_to_array(toks_init).untag("batch").tag("solutions")
xent_min = 10
xent_max = 100
weights = jnp.stack((
    jnp.linspace(-xent_max, -xent_min, MAX_ELITES),
    jnp.ones(MAX_ELITES),
), -1)
@partial(jax.jit, donate_argnums=(0, 1))
def combine_solutions(best_metrics, best, metrics, solutions):
    if best_metrics is not None:
        best_metrics = jnp.concatenate((best_metrics, metrics), 0)
        best = pz.nx.nmap(lambda a, b: jnp.concatenate((a, b)))(
            best.untag("solutions"),
            pz.nx.wrap(solutions, "solutions", "seq").untag("solutions")
        ).tag("solutions").unwrap("solutions", "seq")
    else:
        best_metrics = metrics
        best = solutions
    elite_mask = (best_metrics[None, :] * weights[:, None]).sum(-1).argmax(1)
    best_metrics = best_metrics[elite_mask]
    best = pz.nx.wrap(best[elite_mask], "solutions", "seq")
    return best_metrics, best
for seed in (bar := trange(2_500)):
    solutions, metrics = algo_iteration(best, vector, seed=seed)
    best_metrics, best = combine_solutions(best_metrics, best, metrics, solutions)
    m = {}
    for index in range(MAX_ELITES):
        i = index
        m |= {f"decoded.{i}": tokenizer.decode(best[{"solutions": index}].unwrap("seq")),
              f"loss.{i}": best_metrics[index][0], f"score.{i}": best_metrics[index][1]}
    bar.set_postfix(**m)

Seed: 1034250694


  0%|          | 0/2500 [00:00<?, ?it/s]